In [23]:
import pandas as pd
import io
from io import StringIO

In [2]:
import catboost

In [3]:
import psycopg2

In [27]:
conn_string = "host='172.18.0.2' dbname='postgres' user='postgres' password='postgres'"
conn = psycopg2.connect(conn_string)

In [28]:
cursor = conn.cursor()
df = pd.read_csv('marketing_campaign.csv', sep = '\t')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [25]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres@172.18.0.2:5432/postgres', pool_pre_ping=True)

In [26]:
def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

df.to_sql('marketing_campaign', engine, method=psql_insert_copy)

OperationalError: (psycopg2.OperationalError) connection to server at "172.18.0.2", port 5432 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [19]:
df.to_sql('marketing_campaign', engine)

OperationalError: (psycopg2.OperationalError) connection to server at "172.18.0.2", port 5432 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [18]:
df_pg = pd.read_sql_query('select * from marketing_campaign',con=engine)
df_pg.head()

OperationalError: (psycopg2.OperationalError) connection to server at "172.18.0.2", port 5432 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
cursor.execute('drop table wine')
cursor.execute('commit')
df.to_sql('wine', engine)
df_pg = pd.read_sql_query('select * from wine',con=engine)
df_pg.head()